In [ ]:

import numpy as np
import torch
import time
import torch.nn as nn
import os

In [2]:
##### Import : Vocab and data
import pandas as pd
import pickle

In [3]:
import pickle

with open('./vocab2_new.pkl', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    vocab2 = pickle.load(f)

In [5]:
#2. New option d138 5y onset 100% all
# all data for Topic modeling #/home/skbae/Desktop/146/cancer_conquerance/BEHRT/BEHRT/preprocess/BEHRT_datapreprocess_Aug23_NDP2.ipynb


# import pickle
with open('../preprocess/T20_BFC_ArtrialF_all_100p_summary_F3.pkl',  'rb') as f:     
    dta = pickle.load(f)
    sample_f_all_5y_onset_138 = pd.DataFrame(dta) 

# with open('/home/skbae

In [6]:
# d138 all  : sample_f_all_5y_onset_138

# g_data=sample_f_all_5y_onset_138 

g_data=sample_f_all_5y_onset_138

In [7]:
# NDP_class
group_data_sickFinal_ndp=g_data # pd.DataFrame(g_data[g_data['fold2']==0]).reset_index()

In [10]:
sample_f_NDP=group_data_sickFinal_ndp
sample_f_NDP=sample_f_NDP.reset_index()

In [17]:
## create word dictionary
word_dict = {'PAD': 0, 'CLS': 1, 'SEP': 2, 'MASK': 3, 'UNK' :4 }
for i, w in enumerate(vocab2):
   word_dict[w] = i + 4
   number_dict = {i: w for i, w in enumerate(word_dict)}
   vocab_size = len(word_dict)

len(word_dict)

103

In [15]:
#data = pd.read_parquet(file_config['data'])
data = sample_f_NDP
# remove patients with visits less than min visit
# data['length'] = data['disease_sequence2'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
# data = data[data['length']] >= 3
data = data.reset_index(drop=True)

In [16]:
data.columns

Index(['index', 'ID', 'first_starting_year', 'end_year',
       'specific_disease_year', 'specific_disease_year2', 'disease_sequenceF',
       'length', 'd138', 'response_disease', 'age2_sequence', 'age2_sequenceF',
       'age_B', 'seq_starting_year', 'duration', 'year'],
      dtype='object')

In [18]:
#pop : data_pop.pkl :/home/skbae/Documents/skbae/Cancer_Conquest/BFC.pkl
with open('../../../../BFC.pkl', 'rb') as f: #/home/skbae/cancer_conquerance/BFC.pkl
    data_g = pickle.load(f)
BFC = pd.DataFrame(data_g)

In [19]:
data_BFC=pd.merge(data , BFC, how='left', on='ID')

In [26]:
import pandas as pd

# Convert 'age_B' column to integers (if possible)
data_BFC['age_B'] = pd.to_numeric(data_BFC['age_B'], errors='coerce')

# Age distribution bins and labels
bins = [0, 20, 40, 65, 110]
labels = ['below 20', '20s-30s', '40s-64s', 'over 65s']

# Cut the data into age groups
data_BFC['AgeGroup'] = pd.cut(data_BFC['age_B'], bins=bins, labels=labels, right=False)

# Print the value counts of AgeGroups
print(data_BFC['AgeGroup'].value_counts())

40s-64s     101713
over 65s     47708
20s-30s      20865
below 20        31
Name: AgeGroup, dtype: int64


In [27]:
import numpy as np
data_BFC_table4 =pd.pivot_table(data_BFC, values='ID', index=['d138', 'SEX'],
                       columns=['AgeGroup'], aggfunc=np.size, fill_value=0)

data_BFC_table4 

AgeGroup  below 20  20s-30s  40s-64s  over 65s
d138 SEX                                      
0    1          14     9748    48687     18225
     2          17    11067    51521     26852
1    1           0       39      931      1158
     2           0       11      574      1473

In [28]:
import pandas as pd
import numpy as np

# Assuming you have already created the pivot table data_BFC_table4
data_BFC_table4_percentage = data_BFC_table4.apply(lambda x: (x / x.sum()) * 100, axis=1)

# Create pivot tables for d138 value 0 and 1
data_BFC_table4_d138_0 = data_BFC_table4_percentage[data_BFC_table4_percentage.index.get_level_values('d138') == 0]
data_BFC_table4_d138_1 = data_BFC_table4_percentage[data_BFC_table4_percentage.index.get_level_values('d138') == 1]

# Now you have two separate pivot tables for d138 value 0 and 1

In [29]:
data_BFC_table4_d138_0

AgeGroup  below 20    20s-30s    40s-64s   over 65s
d138 SEX                                           
0    1    0.018259  12.713567  63.498709  23.769466
     2    0.019004  12.371307  57.593034  30.016656

In [30]:
data_BFC_table4_d138_1

AgeGroup  below 20   20s-30s    40s-64s   over 65s
d138 SEX                                          
1    1         0.0  1.832707  43.750000  54.417293
     2         0.0  0.534500  27.891156  71.574344

In [31]:
from scipy.stats import chi2_contingency
# Perform chi-squared test
chi2, p, dof, expected = chi2_contingency(data_BFC_table4_d138_0.values, data_BFC_table4_d138_1.values)

# Print results
print("Chi-squared statistic:", chi2)
print("P-value:", p)

if p < 0.05:
    print("The difference between the groups is statistically significant.")
else:
    print("The difference between the groups is not statistically significant.")

Chi-squared statistic: 1.0183093197875766
P-value: 0.7968217601680717
The difference between the groups is not statistically significant.


In [36]:
# all
# with open('./D138_all_5y_onset_data_BFC2.pkl', 'wb') as f:
#     pickle.dump(data_BFC, f)

# 100% all 
with open('./D138_100pall_5y_onset_data_BFC2_dec08.pkl', 'wb') as f:
    pickle.dump(data_BFC, f)

In [37]:
# #all
# grouped_counts.to_csv('./d138_5y_onset_grouped_all_counts.csv')

#100p all
grouped_counts.to_csv('./d138_5y_onset_1000pall_grouped_counts_dec08.csv') 

BERTTOpic

In [ ]:
# Embedding for BERTTOpic

In [32]:
#import packages
import pandas as pd 
import numpy as np
from bertopic import BERTopic

2024-08-06 13:06:05.055189: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 13:06:05.084246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 13:06:05.575264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-06 13:06:07.301380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [33]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from sklearn.feature_extraction.text import CountVectorizer
#from cuml.cluster import HDBSCAN
#from cuml.manifold import UMAP
from sentence_transformers import SentenceTransformer

Step 1. Load Data

In [34]:
data_BFC.columns

Index(['index', 'ID', 'first_starting_year', 'end_year',
       'specific_disease_year', 'specific_disease_year2', 'disease_sequenceF',
       'length', 'd138', 'response_disease', 'age2_sequence', 'age2_sequenceF',
       'age_B', 'seq_starting_year', 'duration', 'year', 'SEX', 'AGE',
       'GAIBJA', 'AgeGroup'],
      dtype='object')

In [35]:
# Custom function to remove 'SEP' from lists
def remove_sep(lst):
    return [item for item in lst if item != 'SEP']

# Apply the custom function to the column
data_BFC['disease_sequenceF_nosep'] = data_BFC['disease_sequenceF'].apply(remove_sep)
data_BFC.head()

,index,ID,first_starting_year,end_year,specific_disease_year,specific_disease_year2,disease_sequenceF,length,d138,response_disease,...,age2_sequenceF,age_B,seq_starting_year,duration,year,SEX,AGE,GAIBJA,AgeGroup,disease_sequenceF_nosep
0,0,1,2006-01-01,2019-01-01,2011-01-01,2011-01-01,"[219, SEP, 219, SEP, 221, SEP, 219]",3,0,[221],...,"[55, 55, 56, 56, 58, 58, 59]",55,2006-01-01,NaN,NaN,2,51,2,40s-64s,"[219, 219, 221, 219]"
1,1,3,2006-01-01,2019-01-01,2011-01-01,2011-01-01,"[219, SEP, 147, 219, SEP, 147, 219, SEP, 147, ...",3,0,"[145, 147, 219, 220]",...,"[75, 75, 76, 76, 76, 78, 78, 78, 79, 79]",75,2006-01-01,NaN,NaN,2,71,7,over 65s,"[219, 147, 219, 147, 219, 147, 220]"
2,2,4,2002-01-01,2019-01-01,2007-01-01,2007-01-01,"[151, SEP, 151, SEP, 151, SEP, 151, 220, SEP, ...",4,0,[151],...,"[51, 51, 52, 52, 53, 53, 54, 54, 54, 55]",51,2002-01-01,NaN,NaN,1,51,1,40s-64s,"[151, 151, 151, 151, 220, 151]"
3,5,8,2008-01-01,2013-01-01,2013-01-01,2013-01-01,"[220, SEP, 169, SEP, 169, SEP, 134]",3,0,[134],...,"[76, 76, 77, 77, 78, 78, 80]",76,2008-01-01,NaN,NaN,2,70,1,over 65s,"[220, 169, 169, 134]"
4,6,9,2007-01-01,2012-01-01,2012-01-01,2012-01-01,"[220, 236, SEP, 220, 225, SEP, 200, 225]",2,0,"[220, 221]",...,"[44, 44, 44, 45, 45, 45, 48, 48]",44,2007-01-01,NaN,NaN,2,39,6,40s-64s,"[220, 236, 220, 225, 200, 225]"


In [36]:
#load data 
import pandas as pd 
 

#df = test[['ID','d2']]
#df2 =data_BFC[['ID','disease_sequence3','disease_sequence2','age_sequence','d_138', 'Diag_d','SEX','AGE','AgeGroup']] #'cancer']]
df2 =data_BFC[['ID','disease_sequenceF','disease_sequenceF_nosep','age2_sequenceF','length','d138','SEX','AGE','AgeGroup','GAIBJA']] 
#'disease_sequenceF',  'age2_sequenceF', 'length', 'd138'

In [37]:
#Male
df2_m=df2[df2['SEX']==1]
# Female
df2_f=df2[df2['SEX']==2]
print(len(df2_m))
print(len(df2_f))
df2.SEX.value_counts()

78802
91515


2    91515
1    78802
Name: SEX, dtype: int64

In [38]:
df2_m = df2_m.reset_index(drop=True)
df2_f = df2_f.reset_index(drop=True)

In [44]:
# Create a new column by combining AgeGroup, d_138, and SEX columns
#df2['Combined_Column'] = df2['AgeGroup'].astype(str)+ '_' + df2['d_38'].astype(str) + '_' + df2['SEX'].astype(str)
df2['Combined_Column'] = df2['AgeGroup'].astype(str)+ '_' + df2['d138'].astype(str) + '_' + df2['SEX'].astype(str)
# Print the DataFrame with the new column
df2.head()

,ID,disease_sequenceF,disease_sequenceF_nosep,age2_sequenceF,length,d138,SEX,AGE,AgeGroup,GAIBJA,Combined_Column
0,1,"[219, SEP, 219, SEP, 221, SEP, 219]","[219, 219, 221, 219]","[55, 55, 56, 56, 58, 58, 59]",3,0,2,51,40s-64s,2,40s-64s_0_2
1,3,"[219, SEP, 147, 219, SEP, 147, 219, SEP, 147, ...","[219, 147, 219, 147, 219, 147, 220]","[75, 75, 76, 76, 76, 78, 78, 78, 79, 79]",3,0,2,71,over 65s,7,over 65s_0_2
2,4,"[151, SEP, 151, SEP, 151, SEP, 151, 220, SEP, ...","[151, 151, 151, 151, 220, 151]","[51, 51, 52, 52, 53, 53, 54, 54, 54, 55]",4,0,1,51,40s-64s,1,40s-64s_0_1
3,8,"[220, SEP, 169, SEP, 169, SEP, 134]","[220, 169, 169, 134]","[76, 76, 77, 77, 78, 78, 80]",3,0,2,70,over 65s,1,over 65s_0_2
4,9,"[220, 236, SEP, 220, 225, SEP, 200, 225]","[220, 236, 220, 225, 200, 225]","[44, 44, 44, 45, 45, 45, 48, 48]",2,0,2,39,40s-64s,6,40s-64s_0_2


In [41]:
# export 
# Export the DataFrames to pickle files
df2_m.to_pickle('./df2_m.pkl')
df2_f.to_pickle('./df2_f.pkl')


In [3]:
##### Import : Vocab and data
import pandas as pd
import pickle

In [4]:
# import pickle
# Import the DataFrames back from the pickle files
df2_m = pd.read_pickle('./df2_m.pkl')
df2_f = pd.read_pickle('./df2_f.pkl')

In [5]:
#Male
# df2_m=df2[df2['SEX']==1]
texts_m = df2_m['disease_sequenceF_nosep']
# Female
# df2_f=df2[df2['SEX']==2]
texts_f = df2_f['disease_sequenceF_nosep']


In [6]:
#Male
# df2_m=df2[df2['SEX']==1]
# texts_m = df2_m['disease_sequenceF_nosep']
docs_m = df2_m.disease_sequenceF_nosep.apply(str).to_list()
# Female
# df2_f=df2[df2['SEX']==2]
# texts_f = df2_f['disease_sequenceF_nosep']
docs_f = df2_f.disease_sequenceF_nosep.apply(str).to_list()

In [ ]:

classes2_m= df2_m['d138']
classes2_f= df2_f['d138']


In [ ]:
#1. option1 

In [49]:
embeddings.shape # (8066, 384)

(170317, 384)

In [8]:
#male
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model_m = SentenceTransformer("all-MiniLM-L6-v2")
embeddings_m = embedding_model_m.encode(texts_m, show_progress_bar=True)

Batches:   0%|          | 0/2463 [00:00<?, ?it/s]

In [9]:
#female
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model_f = SentenceTransformer("all-MiniLM-L6-v2")
embeddings_f = embedding_model_f.encode(texts_f, show_progress_bar=True)

Batches:   0%|          | 0/2860 [00:00<?, ?it/s]

In [44]:
#Preventing Stochastic Behavior¶

from umap import UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)


In [45]:
# Controlling Number of Topics
from hdbscan import HDBSCAN

#option1
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# # option2
# hdbscan_model = HDBSCAN(min_cluster_size=150,min_samples=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)



In [46]:
# from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer()
#topic_model = BERTopic(ctfidf_model=ctfidf_model )

In [ ]:
## Additional Representations¶

import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
#pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)


# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    #"POS": pos_model
}

In [ ]:
# 1. Male

In [51]:
# male :embeddings_m 

from bertopic import BERTopic

topic_model_m = BERTopic(

  # Pipeline models
  embedding_model=embedding_model_m,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  ctfidf_model = ctfidf_model,
  # vectorizer_model=vectorizer_model,
  # representation_model=representation_model,
  min_topic_size=10,
  # Hyperparameters
  top_n_words=20,    # 10 option2
  verbose=True,
  nr_topics="auto",
calculate_probabilities=True).fit(docs_m, embeddings_m)


topics, probs = topic_model_m.fit_transform(docs_m, embeddings_m)
# topic_model_m.get_topic_info()

# Show topics :topic_model.get_topic_info()
topic_info_138_merged_m = topic_model_m.get_topic_info()


# 100p all option1
# topic_info_138_merged.to_csv('./topic_info_138_100pall_5y_onset2_op1_dec08.csv')
# topic_info_138_merged_m.to_csv('./topic_info_138_100pall_5y_onset2_op1_dec20_male.csv')

# topic_model_m.save("./my_topics_model_138_100pall_5y_onset1_op1_dec20_male")

topic_info_138_merged_m.to_csv('./topic_info_138_100pall_5y_onset2_op1_dec20_aug02male.csv')

topic_model_m.save("./my_topics_model_138_100pall_5y_onset1_op1_dec20_aug02_male")


2024-08-06 13:09:15,233 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-06 13:11:41,368 - BERTopic - Dimensionality - Completed ✓
2024-08-06 13:11:41,369 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

In [ ]:
# 2. Female

In [70]:
# female :embeddings_f

from bertopic import BERTopic

topic_model_f = BERTopic(

  # Pipeline models
  embedding_model=embedding_model_f,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  ctfidf_model = ctfidf_model,
  # vectorizer_model=vectorizer_model,
  # representation_model=representation_model,
  min_topic_size=10,
  # Hyperparameters
  top_n_words=20,    # 10 option2
  verbose=True,
  nr_topics="auto",
calculate_probabilities=True).fit(docs_f, embeddings_f)


topics, probs = topic_model_f.fit_transform(docs_f, embeddings_f)
# topic_model_m.get_topic_info()

# Show topics :topic_model.get_topic_info()
topic_info_138_merged_f = topic_model_f.get_topic_info()


# 100p all option1
# topic_info_138_merged.to_csv('./topic_info_138_100pall_5y_onset2_op1_dec08.csv')
# topic_info_138_merged_f.to_csv('./topic_info_138_100pall_5y_onset2_op1_dec20_female.csv')

# topic_model_f.save("./my_topics_model_138_100pall_5y_onset1_op1_dec20_female")

topic_info_138_merged_f.to_csv('./topic_info_138_100pall_5y_onset2_op1_dec20_aug02_female.csv')

topic_model_f.save("./my_topics_model_138_100pall_5y_onset1_op1_dec20_aug02_female")

2024-08-02 14:10:57,908 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-02 14:13:07,357 - BERTopic - Dimensionality - Completed ✓
2024-08-02 14:13:07,359 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

In [52]:
topic_model_m.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,22333,-1_142_229_134_236,"[142, 229, 134, 236, 239, 238, 225, 143, 160, ...","[['228', '225', '236', '225', '142', '229', '2..."
1,0,9589,0_195_197_229_239,"[195, 197, 229, 239, 225, 198, 145, 228, 221, ...","[['195', '195', '195', '195', '195'], ['195', ..."
2,1,4008,1_151_152_162_220,"[151, 152, 162, 220, 221, 202, 169, 150, 219, ...","[['151', '151', '152', '152'], ['151', '152', ..."
3,2,3327,2_220_232_152_162,"[220, 232, 152, 162, 151, 145, 235, 225, 221, ...","[['151', '220', '220', '152', '220', '151', '1..."
4,3,2720,3_147_145_239_202,"[147, 145, 239, 202, 220, 219, 152, 148, 225, ...","[['145', '147', '147', '145', '145', '147', '1..."
...,...,...,...,...,...
66,65,167,65_136_133_137_132,"[136, 133, 137, 132, 202, 115, 153, 220, 201, ...","[['133', '136', '133', '136', '133', '136', '1..."
67,66,164,66_239_195_197_140,"[239, 195, 197, 140, 241, 143, 228, 134, 229, ...","[['195', '239', '195', '195'], ['195', '239', ..."
68,67,164,67_239_195_229_241,"[239, 195, 229, 241, 238, 197, 201, 200, 220, ...","[['239', '195', '239', '195', '195'], ['239', ..."
69,68,157,68_122_110_154_120,"[122, 110, 154, 120, 236, 105, 114, 147, 150, ...","[['122', '122', '122', '122', '122'], ['122', ..."


In [71]:
topic_model_f.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,20581,-1_169_229_200_238,"[169, 229, 200, 238, 225, 143, 205, 197, 150, ...","[['162', '168', '168', '169', '220', '221', '2..."
1,0,11059,0_218_219_239_221,"[218, 219, 239, 221, 220, 152, 145, 151, 162, ...","[['218', '219', '219', '219'], ['219', '219', ..."
2,1,5842,1_195_197_200_239,"[195, 197, 200, 239, 238, 219, 228, 221, 220, ...","[['195', '195', '195', '195', '195'], ['195', ..."
3,2,4875,2_126_142_157_234,"[126, 142, 157, 234, 236, 150, 199, 160, 102, ...","[['142', '151', '236', '221'], ['142', '151', ..."
4,3,4513,3_152_151_162_145,"[152, 151, 162, 145, 200, 169, 220, 221, 168, ...","[['152', '152', '152', '151', '152'], ['152', ..."
...,...,...,...,...,...
70,69,157,69_229_116_232_236,"[229, 116, 232, 236, 168, 225, 231, 228, 205, ...","[['229', '229', '229', '229', '229'], ['229', ..."
71,70,156,70_147_195_197_229,"[147, 195, 197, 229, 238, 239, 219, 117, 220, ...","[['147', '195', '195'], ['147', '195', '195'],..."
72,71,154,71_200_151_162_142,"[200, 151, 162, 142, 152, 205, 239, 145, 201, ...","[['200', '151', '200', '151'], ['200', '151', ..."
73,72,153,72_133_195_197_239,"[133, 195, 197, 239, 238, 137, 126, 198, 161, ...","[['133', '195', '133', '195', '133', '195'], [..."


In [ ]:
#topic by class and class2

In [53]:
# male

# classes2_m= df2_m['d138']
# classes2_f= df2_f['d138']

topics_per_class_m = topic_model_m.topics_per_class(docs_m, classes=classes2_m)

2it [00:00, 11.05it/s]


In [75]:
# Female

# classes2_m= df2_m['d138']
# classes2_f= df2_f['d138']

topics_per_class_f = topic_model_f.topics_per_class(docs_f, classes=classes2_f)

2it [00:00,  8.72it/s]


In [54]:
topics_per_class_m.head()

,Topic,Words,Frequency,Class
0,-1,"142, 229, 236, 134, 239",21632,0
1,0,"195, 197, 229, 239, 225",9437,0
2,1,"151, 152, 162, 220, 221",3963,0
3,2,"220, 232, 152, 162, 151",3229,0
4,3,"147, 145, 239, 202, 220",2646,0


In [55]:
topics_per_class_m.Class.value_counts()

0    71
1    69
Name: Class, dtype: int64

In [56]:
topic_model_m.visualize_topics_per_class(topics_per_class_m, top_n_topics=10)

In [57]:
table =pd.pivot_table(topics_per_class_m, values='Frequency', index=['Topic', 'Words'],
                       columns=['Class'], aggfunc=np.sum, fill_value=0)
table

Class                              0    1
Topic Words                              
-1    132, 134, 238, 202, 239      0  701
      142, 229, 236, 134, 239  21632    0
 0    195, 197, 229, 239, 225   9437  152
 1    151, 152, 162, 220, 221   3963    0
      152, 151, 246, 162, 110      0   45
...                              ...  ...
 67   239, 229, 219, 220, 195      0    1
 68   122, 110, 154, 120, 236    152    0
      122, 241, 140, 235, 228      0    5
 69   229, 155, 232, 220, 202      0    1
      229, 232, 236, 200, 228    150    0

[139 rows x 2 columns]

In [58]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns

In [59]:
# Create a contingency table
contingency_table = pd.pivot_table(topics_per_class_m, values='Frequency', index='Topic', columns='Class', aggfunc=sum, fill_value=0)

# Perform the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-Square Statistic:", chi2_stat)
print("P-value:", p_val)
print("Degrees of Freedom:", dof)
#print("Expected Frequencies:\n", expected)

#Interpret the results
alpha = 0.05  # significance level

if p_val < alpha:
    print("Reject the null hypothesis: The variables are dependent.")
else:
    print("Fail to reject the null hypothesis: The variables are independent.")

Chi-Square Statistic: 439.9944971699995
P-value: 5.022502995364892e-55
Degrees of Freedom: 70
Reject the null hypothesis: The variables are dependent.


In [60]:
# Show topics
topics_per_class_info_m = topic_model_m.get_topic_info()

In [61]:
topics_per_class_info_m.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,22333,-1_142_229_134_236,"[142, 229, 134, 236, 239, 238, 225, 143, 160, ...","[['228', '225', '236', '225', '142', '229', '2..."
1,0,9589,0_195_197_229_239,"[195, 197, 229, 239, 225, 198, 145, 228, 221, ...","[['195', '195', '195', '195', '195'], ['195', ..."
2,1,4008,1_151_152_162_220,"[151, 152, 162, 220, 221, 202, 169, 150, 219, ...","[['151', '151', '152', '152'], ['151', '152', ..."
3,2,3327,2_220_232_152_162,"[220, 232, 152, 162, 151, 145, 235, 225, 221, ...","[['151', '220', '220', '152', '220', '151', '1..."
4,3,2720,3_147_145_239_202,"[147, 145, 239, 202, 220, 219, 152, 148, 225, ...","[['145', '147', '147', '145', '145', '147', '1..."


In [62]:
# visualize terms

topic_model_m.visualize_barchart()#top_n_topics=16)

In [63]:
topics_per_class2_m = topic_model_m.topics_per_class(docs_m, classes=classes2_m)
topics_per_class2_m.Class.value_counts()

2it [00:00,  9.78it/s]


0    71
1    69
Name: Class, dtype: int64

In [64]:
topics_per_class_m.Class.value_counts()

0    71
1    69
Name: Class, dtype: int64

In [65]:
# topics_per_class2_m.to_csv('./topics_per_d138_100pall_5y_onset1_2_op1_dec20_by138_male.csv') # 100p 
topics_per_class_m.to_csv('./topics_per_d138_100pall_5y_onset1_2_op1_dec20_aug02_by138_male.csv') # 100p 

In [ ]:
# female

In [ ]:
topics_per_class_info_f.head()

In [ ]:
topic_model_f.visualize_barchart()#top_n_topics=16)

In [79]:
topics_per_class2_f = topic_model_f.topics_per_class(docs_f, classes=classes2_f)
# topics_per_class2_f.to_csv('./topics_per_d138_100pall_5y_onset1_2_op1_dec20_by138_female.csv') # 100p 
topics_per_class2_f.to_csv('./topics_per_d138_100pall_5y_onset1_2_op1_dec20_aug02_by138_female.csv') # 100p 
topics_per_class2_f.Class.value_counts()

2it [00:00,  9.04it/s]


0    75
1    71
Name: Class, dtype: int64

In [ ]:
# add topic model graph figure Aug08

In [ ]:
# upload the model again 

In [10]:
from bertopic import BERTopic

# Load the saved models
topic_model_m = BERTopic.load("./my_topics_model_138_100pall_5y_onset1_op1_dec20_aug02_male")
topic_model_f = BERTopic.load("./my_topics_model_138_100pall_5y_onset1_op1_dec20_aug02_female") #./my_topics_model_138_100pall_5y_onset1_op1_dec20_aug02_female


2024-08-29 16:34:17.274790: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 16:34:17.713823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 16:34:22.035395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-29 16:34:28.706675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [12]:
topic_model_f.visualize_barchart()#top_n_topics=16)

In [3]:
# topic_model_f.visualize_barchart()#top_n_topics=16)

In [11]:
topic_model_m.visualize_barchart()#top_n_topics=16)

In [4]:
topic_model_m.visualize_barchart()#top_n_topics=16)

In [ ]:
# classes2_m= df2_m['d138']
# classes2_f= df2_f['d138']

In [47]:
import pandas as pd
from bertopic import BERTopic


# Get the number of topics in the model
num_topics = topic_model_m.get_topic_info().shape[0] - 1 

# Create an empty list to store all topic data
all_topic_data = []

# Analyze all topics
for topic_num in range(num_topics):

    # Get the topic representation for the current topic
    topic_words = topic_model_m.get_topic(topic_num)

    # Convert the topic representation to a DataFrame
    topic_words_df = pd.DataFrame(topic_words, columns=['Word', 'c-TF-IDF'])

    # Add a 'Topic' column to identify the topic number
    topic_words_df['Topic'] = topic_num

    # Append the DataFrame to the list
    all_topic_data.append(topic_words_df)

# Concatenate all DataFrames into a single DataFrame
all_topics_df_m = pd.concat(all_topic_data, ignore_index=True)

# Export to CSV
all_topics_df_m.to_csv('./all_topics_output_m_Aug29.csv', index=False)

# print("All topics data exported to all_topics_output.csv")

In [48]:
import pandas as pd
from bertopic import BERTopic


# Get the number of topics in the model
num_topics = topic_model_f.get_topic_info().shape[0] - 1 

# Create an empty list to store all topic data
all_topic_data = []

# Analyze all topics
for topic_num in range(num_topics):

    # Get the topic representation for the current topic
    topic_words = topic_model_f.get_topic(topic_num)

    # Convert the topic representation to a DataFrame
    topic_words_df = pd.DataFrame(topic_words, columns=['Word', 'c-TF-IDF'])

    # Add a 'Topic' column to identify the topic number
    topic_words_df['Topic'] = topic_num

    # Append the DataFrame to the list
    all_topic_data.append(topic_words_df)

# Concatenate all DataFrames into a single DataFrame
all_topics_df_f = pd.concat(all_topic_data, ignore_index=True)

# Export to CSV
all_topics_df_f.to_csv('./all_topics_output_m_Aug29.csv', index=False)

In [50]:
all_topics_df_m

,Word,c-TF-IDF,Topic
0,195,0.069761,0
1,197,0.019832,0
2,229,0.008072,0
3,239,0.006522,0
4,225,0.005092,0
...,...,...,...
1395,199,0.005555,69
1396,231,0.005518,69
1397,145,0.005370,69
1398,235,0.005129,69


In [49]:
all_topics_df_f

,Word,c-TF-IDF,Topic
0,218,0.045629,0
1,219,0.044528,0
2,239,0.010866,0
3,221,0.010649,0
4,220,0.009945,0
...,...,...,...
1475,205,0.006148,73
1476,221,0.006098,73
1477,169,0.005953,73
1478,228,0.005145,73


In [51]:
topics_per_class2_m.Class.value_counts()

0    71
1    69
Name: Class, dtype: int64

In [52]:
topics_per_class2_m.head()

,Topic,Words,Frequency,Class
0,-1,"142, 229, 236, 134, 239",21632,0
1,0,"195, 197, 229, 239, 225",9437,0
2,1,"151, 152, 162, 220, 221",3963,0
3,2,"220, 232, 152, 162, 151",3229,0
4,3,"147, 145, 239, 202, 220",2646,0


In [ ]:
### Disease codem maping######

In [ ]:

# # 100% all 
# with open('./D138_100pall_5y_onset_data_BFC2_dec08.pkl', 'wb') as f:
#     pickle.dump(data_BFC, f)

In [53]:
# Import the necessary libraries
import pandas as pd
# Reload the datasets
#optino1
# data_all = pd.read_csv("../topics_per_class2_50pall_5y_onset1_2_oct27_F.csv") # /home/skbae/Documents/skbae/Cancer_Conquest/BEHRT/BEHRT/BEHRT/disease_codes.xlsx /home/skbae/Documents/skbae/Cancer_Conquest/BEHRT/BEHRT/BEHRT/topics_per_class2_50pall_5y_onset1_2_oct27_F.csv

#option2 :6Y
# data_all = pd.read_csv('./topics_per_d138_100pall_5y_onset1_2_op2_nov07.csv') #
# data_all = pd.read_csv('./topics_per_d138_100pall_5y_onset1_2_op1_dec20_by138.csv') #
# data_sample = pd.read_csv("/mnt/data/d138_50p_2t_sample_bertopic.csv")
disease_codes = pd.read_excel("../disease_codes.xlsx")

In [54]:
disease_codes

,Words,dname,wash_out,dname.1,Unnamed: 4,Unnamed: 5
0,101,Esophageal cancer,5,식도암,NaN,NaN
1,102,Stomach cancer,5,위암,87.0,NaN
2,103,Liver cancer,5,간 암,NaN,NaN
3,104,Larynx cancer,5,후두암,NaN,NaN
4,105,"Trachea, bronchus and lung cancers",5,"기관, 기관지 및 폐암",NaN,NaN
...,...,...,...,...,...,...
121,239,Cataracts,5,백내장,NaN,NaN
122,240,Macular degeneration,5,황반 변성,NaN,NaN
123,241,Refraction and accommodation disorders,5,굴절 및 조절 장애,NaN,NaN
124,245,Dental caries,5,충치,NaN,NaN


In [ ]:
data_BFC.columns

Index(['index', 'ID', 'first_starting_year', 'end_year',
       'specific_disease_year', 'specific_disease_year2', 'disease_sequenceF',
       'length', 'd138', 'response_disease', 'age2_sequence', 'age2_sequenceF',
       'age_B', 'seq_starting_year', 'duration', 'year', 'SEX', 'AGE',
       'GAIBJA', 'AgeGroup'],
      dtype='object')

In [ ]:
data_all.Class.value_counts()

0    140
1    138
Name: Class, dtype: int64

In [57]:
#male : Aug05
data_all_m=topics_per_class2_m
data_all_m.Class.value_counts()

0    71
1    69
Name: Class, dtype: int64

In [58]:
# female : AUg05
data_all_f=topics_per_class2_f
data_all_f.Class.value_counts()

0    75
1    71
Name: Class, dtype: int64

In [59]:
# Create a mapping from 'ords' to 'dname'
disease_mapping = disease_codes.set_index('Words')['dname'].to_dict()

# Define a function to map codes to disease names
def map_codes_to_diseases(codes):
    # Split the codes into a list
    codes = codes.split(", ")
    # Map each code to its disease name
    diseases = [disease_mapping.get(int(code), "Unknown") for code in codes]
    # Join the disease names back into a string
    return ", ".join(diseases)

# Apply the function to the 'Words' column
# topics_per_gender['Disease Names'] = topics_per_gender['Words'].apply(map_codes_to_diseases)

In [62]:
#male

# Mapping disease codes for the sample dataset
data_all_m['Disease Names'] = data_all_m['Words'].apply(map_codes_to_diseases)


In [70]:
disease_codes[disease_codes.Words==152]

,Words,dname,wash_out,dname.1,Unnamed: 4,Unnamed: 5
47,152,Gastritis and duodenitis,5,위염 및 십이지장염,NaN,NaN


In [64]:
data_all_m.head()

,Topic,Words,Frequency,Class,Disease Names
0,-1,"142, 229, 236, 134, 239",21632,0,"Hermorrhoid, Abscess, impetigo, and other bact..."
1,0,"195, 197, 229, 239, 225",9437,0,"Diabetes mellitus, Chronic kidney disease due ..."
2,1,"151, 152, 162, 220, 221",3963,0,"Peptic ulcer disease, Gastritis and duodenitis..."
3,2,"220, 232, 152, 162, 151",3229,0,"Low back pain, Viral skin diseases, Gastritis ..."
4,3,"147, 145, 239, 202, 220",2646,0,"Asthma, Chronic obstructive pulmonary disease,..."


In [65]:
#Extracting data for "disease 138 yes" and "disease 138 no" groups from the all data given file
all_data_m_yes = data_all_m[data_all_m['Class'] == 1] # 138 yes
all_data_m_no = data_all_m[data_all_m['Class'] ==0] #'138 no']
# Identify topics and associated diseases for each group
all_data_m_yes_topics_diseases = all_data_m_yes[['Topic', 'Disease Names']].drop_duplicates().reset_index(drop=True)
all_data_m_no_topics_diseases = all_data_m_no[['Topic', 'Disease Names']].drop_duplicates().reset_index(drop=True)
# Display the topics and associated diseases for the "yes" and "no" groups from the all data given file
all_data_m_yes_topics_diseases

,Topic,Disease Names
0,-1,"Rheumatic heart disease, Ischemic stroke, Glau..."
1,0,"Diabetes mellitus, Chronic kidney disease due ..."
2,1,"Gastritis and duodenitis, Peptic ulcer disease..."
3,2,"Low back pain, Chronic obstructive pulmonary d..."
4,3,"Asthma, Chronic obstructive pulmonary disease,..."
...,...,...
64,65,"Hypertensive heart disease, Ischemic heart dis..."
65,66,"Cataracts, Diabetes mellitus, Cellulitis, Chro..."
66,67,"Cataracts, Abscess, impetigo, and other bacter..."
67,68,"Bladder cancer, Refraction and accommodation d..."


In [66]:
all_data_m_no_topics_diseases

,Topic,Disease Names
0,-1,"Hermorrhoid, Abscess, impetigo, and other bact..."
1,0,"Diabetes mellitus, Chronic kidney disease due ..."
2,1,"Peptic ulcer disease, Gastritis and duodenitis..."
3,2,"Low back pain, Viral skin diseases, Gastritis ..."
4,3,"Asthma, Chronic obstructive pulmonary disease,..."
...,...,...
66,65,"Hypertensive heart disease, Ischemic heart dis..."
67,66,"Cataracts, Diabetes mellitus, Chronic kidney d..."
68,67,"Cataracts, Diabetes mellitus, Abscess, impetig..."
69,68,"Bladder cancer, Colon and rectum cancers, Para..."


In [67]:
#5Y option1 :male
# data_all.to_csv('./bertopic_5y_138_100pall_op1_dec08.csv', index=False)
# data_all_m.to_csv('./bertopic_5y_138_100pall_op1_dec20_maleF.csv', index=False)
# data_all_m.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_maleF.csv', index=False)
data_all_m.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_maleF_aug29.csv', index=False)

In [68]:
# female : topics_per_class2_f
# Mapping disease codes for the sample dataset
data_all_f['Disease Names'] = data_all_f['Words'].apply(map_codes_to_diseases)# (map_codes_to_names_v2)

In [69]:
data_all_f.head()

,Topic,Words,Frequency,Class,Disease Names
0,-1,"169, 229, 200, 225, 238",20077,0,"Tension-type headache, Abscess, impetigo, and ..."
1,0,"218, 219, 239, 221, 220",10760,0,"Rheumatoid arthritis, Osteoarthritis, Cataract..."
2,1,"195, 197, 200, 239, 238",5767,0,"Diabetes mellitus, Chronic kidney disease due ..."
3,2,"126, 142, 157, 234, 236",4771,0,"Non-Hodgkins lymphoma, Hermorrhoid, Ulcerative..."
4,3,"152, 151, 162, 145, 200",4456,0,"Gastritis and duodenitis, Peptic ulcer disease..."


In [71]:
data_all_f[data_all_f['Class']==1].head()

,Topic,Words,Frequency,Class,Disease Names
75,-1,"132, 133, 238, 134, 169",504,1,"Rheumatic heart disease, Ischemic heart diseas..."
76,0,"219, 218, 164, 220, 239",299,1,"Osteoarthritis, Rheumatoid arthritis, Alzheime..."
77,1,"195, 200, 232, 237, 239",75,1,"Diabetes mellitus, Tubulointerstitial nephriti..."
78,2,"132, 165, 142, 114, 200",104,1,"Rheumatic heart disease, Parkinsons disease, H..."
79,3,"152, 151, 145, 162, 232",57,1,"Gastritis and duodenitis, Peptic ulcer disease..."


In [77]:


#Extracting data for "disease 138 yes" and "disease 138 no" groups from the all data given file
all_data_f_yes = data_all_f[data_all_f['Class'] == 1] # 138 yes
all_data_f_no = data_all_f[data_all_f['Class'] ==0] #'138 no']
# Identify topics and associated diseases for each group
all_data_f_yes_topics_diseases = all_data_f_yes[['Topic', 'Disease Names']].drop_duplicates().reset_index(drop=True)
all_data_f_no_topics_diseases = all_data_f_no[['Topic', 'Disease Names']].drop_duplicates().reset_index(drop=True)
# Display the topics and associated diseases for the "yes" and "no" groups from the all data given file
all_data_f_yes_topics_diseases

#5Y option1 :female
# data_all.to_csv('./bertopic_5y_138_100pall_op1_dec08.csv', index=False)
# data_all_f.to_csv('./bertopic_5y_138_100pall_op1_dec20_femaleF.csv', index=False)
data_all_f.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_femaleF_aug29.csv', index=False)

In [73]:
data_all_f.head()

,Topic,Words,Frequency,Class,Disease Names
0,-1,"169, 229, 200, 225, 238",20077,0,"Tension-type headache, Abscess, impetigo, and ..."
1,0,"218, 219, 239, 221, 220",10760,0,"Rheumatoid arthritis, Osteoarthritis, Cataract..."
2,1,"195, 197, 200, 239, 238",5767,0,"Diabetes mellitus, Chronic kidney disease due ..."
3,2,"126, 142, 157, 234, 236",4771,0,"Non-Hodgkins lymphoma, Hermorrhoid, Ulcerative..."
4,3,"152, 151, 162, 145, 200",4456,0,"Gastritis and duodenitis, Peptic ulcer disease..."


In [74]:
data_all_f[data_all_f['Class']==1].head()

,Topic,Words,Frequency,Class,Disease Names
75,-1,"132, 133, 238, 134, 169",504,1,"Rheumatic heart disease, Ischemic heart diseas..."
76,0,"219, 218, 164, 220, 239",299,1,"Osteoarthritis, Rheumatoid arthritis, Alzheime..."
77,1,"195, 200, 232, 237, 239",75,1,"Diabetes mellitus, Tubulointerstitial nephriti..."
78,2,"132, 165, 142, 114, 200",104,1,"Rheumatic heart disease, Parkinsons disease, H..."
79,3,"152, 151, 145, 162, 232",57,1,"Gastritis and duodenitis, Peptic ulcer disease..."


In [75]:
all_data_f_yes_topics_diseases

,Topic,Disease Names
0,-1,"Rheumatic heart disease, Ischemic heart diseas..."
1,0,"Osteoarthritis, Rheumatoid arthritis, Alzheime..."
2,1,"Diabetes mellitus, Tubulointerstitial nephriti..."
3,2,"Rheumatic heart disease, Parkinsons disease, H..."
4,3,"Gastritis and duodenitis, Peptic ulcer disease..."
...,...,...
66,68,"Ischemic stroke, Gout, Hypertensive heart dise..."
67,70,"Asthma, Diabetes mellitus, Glaucoma, Tubuloint..."
68,71,"Tubulointerstitial nephritis, pyelonephritis, ..."
69,72,"Ischemic heart disease, Diabetes mellitus, Rhe..."


In [76]:
all_data_f_no_topics_diseases

,Topic,Disease Names
0,-1,"Tension-type headache, Abscess, impetigo, and ..."
1,0,"Rheumatoid arthritis, Osteoarthritis, Cataract..."
2,1,"Diabetes mellitus, Chronic kidney disease due ..."
3,2,"Non-Hodgkins lymphoma, Hermorrhoid, Ulcerative..."
4,3,"Gastritis and duodenitis, Peptic ulcer disease..."
...,...,...
70,69,"Abscess, impetigo, and other bacterial skin di..."
71,70,"Asthma, Diabetes mellitus, Chronic kidney dise..."
72,71,"Tubulointerstitial nephritis, pyelonephritis, ..."
73,72,"Ischemic heart disease, Diabetes mellitus, Chr..."


In [ ]:
# check the differnce test 

In [ ]:
# import the data 

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

# Import the data
# data_all_f = pd.read_csv('./bertopic_5y_138_100pall_op1_dec20_femaleF.csv')
# data_all_m = pd.read_csv('./bertopic_5y_138_100pall_op1_dec20_maleF.csv')

# Aug02
# Import the data
data_all_f = pd.read_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_femaleF.csv')
data_all_m = pd.read_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_maleF.csv')

In [2]:
data_all_f

,Topic,Words,Frequency,Class,Disease Names
0,-1,"169, 229, 200, 225, 238",20077,0,"Tension-type headache, Abscess, impetigo, and ..."
1,0,"218, 219, 239, 221, 220",10760,0,"Rheumatoid arthritis, Osteoarthritis, Cataract..."
2,1,"195, 197, 200, 239, 238",5767,0,"Diabetes mellitus, Chronic kidney disease due ..."
3,2,"126, 142, 157, 234, 236",4771,0,"Non-Hodgkins lymphoma, Hermorrhoid, Ulcerative..."
4,3,"152, 151, 162, 145, 200",4456,0,"Gastritis and duodenitis, Peptic ulcer disease..."
...,...,...,...,...,...
141,68,"134, 222, 136, 162, 168",11,1,"Ischemic stroke, Gout, Hypertensive heart dise..."
142,70,"147, 195, 238, 200, 197",7,1,"Asthma, Diabetes mellitus, Glaucoma, Tubuloint..."
143,71,"200, 151, 148, 229, 154",2,1,"Tubulointerstitial nephritis, pyelonephritis, ..."
144,72,"133, 195, 132, 137, 238",10,1,"Ischemic heart disease, Diabetes mellitus, Rhe..."


In [3]:
data_all_m.columns

Index(['Topic', 'Words', 'Frequency', 'Class', 'Name', 'Disease Names'], dtype='object')

In [4]:
data_all_m[['Topic','Words','Disease Names']]

,Topic,Words,Disease Names
0,-1,"142, 229, 236, 134, 239","Hermorrhoid, Abscess, impetigo, and other bact..."
1,0,"195, 197, 229, 239, 225","Diabetes mellitus, Chronic kidney disease due ..."
2,1,"151, 152, 162, 220, 221","Peptic ulcer disease, Gastritis and duodenitis..."
3,2,"220, 232, 152, 162, 151","Low back pain, Viral skin diseases, Gastritis ..."
4,3,"147, 145, 239, 202, 220","Asthma, Chronic obstructive pulmonary disease,..."
...,...,...,...
135,65,"136, 133, 157, 241, 202","Hypertensive heart disease, Ischemic heart dis..."
136,66,"239, 195, 228, 145, 225","Cataracts, Diabetes mellitus, Cellulitis, Chro..."
137,67,"239, 229, 219, 220, 195","Cataracts, Abscess, impetigo, and other bacter..."
138,68,"122, 241, 140, 235, 228","Bladder cancer, Refraction and accommodation d..."


In [5]:
data_all_f.columns = ['Topic', 'Words', 'Frequency', 'Class', 'Disease']

In [6]:
# Assuming 'data_all_f' is your DataFrame and it has columns named 'Frequency', 'Topic', and 'Class'
# sum_freq = data_all_f.groupby(['Topic', 'Class'])['Frequency'].sum().reset_index()
# sum_freq

# Assuming 'data_all_f' is your DataFrame and it has columns named 'Frequency', 'Topic', and 'Class'
sum_freq = data_all_f[data_all_f['Topic'] != -1].groupby(['Topic', 'Class'])['Frequency'].sum().reset_index()

sum_freq

,Topic,Class,Frequency
0,0,0,10760
1,0,1,299
2,1,0,5767
3,1,1,75
4,2,0,4771
...,...,...,...
139,71,1,2
140,72,0,143
141,72,1,10
142,73,0,143


In [7]:
from scipy.stats import chi2_contingency

# Reshape the data into a contingency table
contingency = sum_freq.pivot(index='Topic', columns='Class', values='Frequency').fillna(0)

# Perform Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi2 = {}, p-value = {}".format(chi2, p))

# Determine if null hypothesis is rejected
if p < 0.05:
    print("Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.")
else:
    print("Null hypothesis is not rejected. There is no significant difference in the frequency distribution by class.")

Chi2 = 780.3404434183128, p-value = 6.18270610853098e-119
Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.


In [8]:
# Assuming 'data_all_f' is your DataFrame and it has columns named 'Frequency', 'Topic', and 'Class'
sum_freq_f = data_all_f[data_all_f['Topic'].isin([0, 1, 2, 3, 4])].groupby(['Topic', 'Class'])['Frequency'].sum().reset_index()

print(sum_freq_f)

   Topic  Class  Frequency
0      0      0      10760
1      0      1        299
2      1      0       5767
3      1      1         75
4      2      0       4771
5      2      1        104
6      3      0       4456
7      3      1         57
8      4      0       4379
9      4      1         83


In [9]:
from scipy.stats import chi2_contingency

# Reshape the data into a contingency table
contingency = sum_freq_f.pivot(index='Topic', columns='Class', values='Frequency').fillna(0)

# Perform Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi2 = {}, p-value = {}".format(chi2, p))

# Determine if null hypothesis is rejected
if p < 0.05:
    print("Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.")
else:
    print("Null hypothesis is not rejected. There is no significant difference in the frequency distribution by class.")

Chi2 = 56.33892979086725, p-value = 1.7024928628369204e-11
Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.


In [10]:
# Assuming 'data_all_m' is your DataFrame and it has columns named 'Frequency', 'Topic', and 'Class'
sum_freq_m = data_all_m[data_all_m['Topic'] != -1].groupby(['Topic', 'Class'])['Frequency'].sum().reset_index()

sum_freq_m

,Topic,Class,Frequency
0,0,0,9437
1,0,1,152
2,1,0,3963
3,1,1,45
4,2,0,3229
...,...,...,...
133,67,1,1
134,68,0,152
135,68,1,5
136,69,0,150


In [11]:
from scipy.stats import chi2_contingency

# Reshape the data into a contingency table
contingency_m = sum_freq_m.pivot(index='Topic', columns='Class', values='Frequency').fillna(0)

# Perform Chi-Square test
chi2_m, p_m, dof_m, expected_m = chi2_contingency(contingency_m)

print("Chi2 = {}, p-value = {}".format(chi2_m, p_m))

# Determine if null hypothesis is rejected
if p_m < 0.05:
    print("Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.")
else:
    print("Null hypothesis is not rejected. There is no significant difference in the frequency distribution by class.")

Chi2 = 445.0285815548638, p-value = 2.3302041107403483e-56
Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.


In [12]:
# Assuming 'data_all_f' is your DataFrame and it has columns named 'Frequency', 'Topic', and 'Class'
sum_freq_m2 = data_all_m[data_all_m['Topic'].isin([0, 1, 2, 3, 4])].groupby(['Topic', 'Class'])['Frequency'].sum().reset_index()

print(sum_freq_m2)

   Topic  Class  Frequency
0      0      0       9437
1      0      1        152
2      1      0       3963
3      1      1         45
4      2      0       3229
5      2      1         98
6      3      0       2646
7      3      1         74
8      4      0       2338
9      4      1         95


In [13]:
from scipy.stats import chi2_contingency

# Reshape the data into a contingency table
contingency_m2 = sum_freq_m2.pivot(index='Topic', columns='Class', values='Frequency').fillna(0)

# Perform Chi-Square test
chi2_m, p_m, dof_m, expected_m = chi2_contingency(contingency_m2)

print("Chi2 = {}, p-value = {}".format(chi2_m, p_m))

# Determine if null hypothesis is rejected
if p_m < 0.05:
    print("Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.")
else:
    print("Null hypothesis is not rejected. There is no significant difference in the frequency distribution by class.")

Chi2 = 86.11889396863475, p-value = 8.781300166671014e-18
Null hypothesis is rejected. There is a significant difference in the frequency distribution by class.


In [14]:
import pandas as pd

# 예시 데이터프레임 생성 (sum_freq_m과 sum_freq_f가 데이터프레임이라고 가정)
# sum_freq_m = pd.DataFrame(...)
# sum_freq_f = pd.DataFrame(...)

# CSV 파일로 내보내기
sum_freq_m.to_csv('./sum_freq_m_aug06.csv', index=False)
sum_freq.to_csv('./sum_freq_f_aug06.csv', index=False)

In [ ]:
# #5Y option1 :female
# # data_all.to_csv('./bertopic_5y_138_100pall_op1_dec08.csv', index=False)
# data_all_f.to_csv('./bertopic_5y_138_100pall_op1_dec20_femaleF.csv', index=False)

# data_all_f.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_femaleF.csv', index=False)
data_all_f.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_femaleF_aug06.csv', index=False)
# #5Y option1 :male
# # data_all.to_csv('./bertopic_5y_138_100pall_op1_dec08.csv', index=False)
# data_all_m.to_csv('./bertopic_5y_138_100pall_op1_dec20_maleF.csv', index=False)
# data_all_m.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_maleF.csv', index=False)
data_all_m.to_csv('./bertopic_5y_138_100pall_op1_dec20_aug02_maleF_aug06.csv', index=False)